## Demo: Word Window Classification

Until this part of the notebook, we have learned the fundamentals of PyTorch and built a basic network solving a toy task. Now we will attempt to solve an example NLP task. Here are the things we will learn:

1. Data: Creating a Dataset of Batched Tensors
2. Modeling
3. Training
4. Prediction

In this section, our goal will be to train a model that will find the words in a sentence corresponding to a `LOCATION`, which will be always of span `1` (meaning that `San Fransisco` won't be recognized as a `LOCATION`). Our task is called `Word Window Classification` for a reason. Instead of letting our model to only take a look at one word in each forward pass, we would like it to be able to consider the context of the word in question. That is, for each word, we want our model to be aware of the surrounding words. Let's dive in!

### Data

The very first task of any machine learning project is to set up our training set. Usually, there will be a training corpus we will be utilizing. In NLP tasks, the corpus would generally be a `.txt` or `.csv` file where each row corresponds to a sentence or a tabular datapoint. In our toy task, we will assume that we have already read our data and the corresponding labels into a `Python` list.

In [12]:
import torch
import torch.nn as nn

# Import pprint, module we use for making our print statements prettier
import pprint
pp = pprint.PrettyPrinter()

In [60]:
# Our raw data, which consists of sentences
# corpus = [
#           "We always come to Paris",
#           "The professor is from Australia",
#           "I live in Stanford",
#           "He comes from Taiwan",
#           "The capital of Turkey is Ankara"
#          ]
import pandas as pd
df = pd.DataFrame(pd.read_csv('raw_sample.csv'))
corpus = list(df['posting'])

#### Preprocessing

To make it easier for our models to learn, we usually apply a few preprocessing steps to our data. This is especially important when dealing with text data. Here are some examples of text preprocessing:
* **Tokenization**: Tokenizing the sentences into words.
* **Lowercasing**: Changing all the letters to be lowercase.
* **Noise removal:** Removing special characters (such as punctuations). 
* **Stop words removal**: Removing commonly used words.

Which preprocessing steps are necessary is determined by the task at hand. For example, although it is useful to remove special characters in some tasks, for others they may be important (for example, if we are dealing with multiple languages). For our task, we will lowercase our words and tokenize. 


In [61]:
# The preprocessing function we will use to generate our training examples
# Our function is a simple one, we lowercase the letters
# and then tokenize the words.
def preprocess_sentence(sentence):
  return sentence.lower().split()

# Create our training set
train_sentences = [sent.lower().split() for sent in corpus]
train_sentences

[['auto',
  ',',
  'sun',
  'roof',
  ',',
  'lane',
  'assist',
  ',',
  'park',
  'assist,',
  'loaded,',
  'all',
  'apps',
  ',',
  'back',
  'up',
  'camera',
  ',',
  'navigation',
  ',',
  'bluetooth',
  ',',
  'super',
  'low',
  'miles',
  'with',
  'carfax',
  'to',
  'back',
  'up',
  ',',
  'non',
  'smoker',
  'super',
  'clean',
  'in',
  'and',
  'out',
  'salvage',
  'do',
  'to',
  'damage',
  'to',
  'rear',
  'bumper',
  'only',
  ',',
  'i',
  'included',
  'a',
  'before',
  'picture',
  'so',
  'u',
  'can',
  'se',
  'for',
  'your',
  'self',
  'car',
  'runs',
  'and',
  'looks',
  'lk',
  'new',
  'and',
  'it',
  'comes',
  'with',
  'a',
  '30',
  'day',
  'warranty',
  'and',
  '3',
  'day',
  'money',
  'back',
  'guaranteed',
  'for',
  'more',
  'info',
  'please',
  'call',
  'show',
  'contact',
  'info',
  '2017',
  'toyota',
  'prius',
  'loadedvin:',
  'jtdkarfu3h3045352',
  'condition:',
  'like',
  'new',
  'cylinders:',
  '4',
  'cylinders',
  'f

For each training example we have, we should also have a corresponding label. Recall that the goal of our model was to determine which words correspond to a `LOCATION`. That is, we want our model to output `0` for all the words that are not `LOCATION`s and `1` for the ones that are `LOCATION`s.

In [64]:
# locations = df['make']

In [66]:
# Set of locations that appear in our corpus
# locations = set(["australia", "ankara", "paris", "stanford", "taiwan", "turkey"])
locations = set(df['make'])
# Our train labels
train_labels = [[1 if word in locations else 0 for word in sent] for sent in train_sentences]
train_labels

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

#### Converting Words to Embeddings

Let's look at our training data a little more closely. Each datapoint we have is a sequence of words. On the other hand, we know that machine learning models work with numbers in vectors. How are we going to turn words into numbers? You may be thinking embeddings and you are right!

Imagine that we have an embedding lookup table `E`, where each row corresponds to an embedding. That is, each word in our vocabulary would have a corresponding embedding row `i` in this table. Whenever we want to find an embedding for a word, we will follow these steps:
1. Find the corresponding index `i` of the word in the embedding table: `word->index`.
2. Index into the embedding table and get the embedding: `index->embedding`.

Let's look at the first step. We should assign all the words in our vocabulary to a corresponding index. We can do it as follows:
1. Find all the unique words in our corpus.
2. Assign an index to each.

In [67]:
# Find all the unique words in our corpus 
vocabulary = set(w for s in train_sentences for w in s)
vocabulary

{'primarily',
 'this',
 'calipers',
 'corvette',
 'pay',
 'engine:',
 'jtdkarfu3h3045352',
 'kidding).',
 'inside',
 'repaired',
 'car.its',
 'stereo',
 'party',
 'boosted,',
 'option',
 '16’',
 'runs',
 'protection',
 '70000',
 'certified',
 'sunroof,',
 'gate',
 'gt',
 '15',
 'plus-pj8:',
 '92800',
 'acdelco',
 'crv',
 'they',
 'little',
 '-front',
 'back-up',
 'timing',
 'moon/sunroof,',
 'bas',
 'interior,',
 'transmission',
 'smog',
 'last',
 'player-satalite',
 '(2000',
 '4',
 'button',
 '68800',
 'range',
 'cloth',
 'dealership,',
 'storage',
 'disk',
 'highest',
 'model:',
 'beauty.',
 'red',
 'warmers,',
 'hot',
 'me',
 'garage.its',
 'ecoboost',
 'color:',
 'comes',
 '(nmo',
 '.',
 'jn8at2mt5gw001233',
 'block',
 'bluetooth',
 "damaged/doesn't",
 's63,',
 '2.5i',
 'certificate',
 "weren't",
 'camaro,',
 'pcm',
 'lifters',
 'push',
 'emissions',
 'departure',
 'deep',
 'carfax,',
 'bag-side-head',
 'cruise',
 '(',
 'style.',
 'my',
 'rain',
 'speed',
 'a/c',
 'wont',
 'twin',


`vocabulary` now contains all the words in our corpus. On the other hand, during the test time, we can see words that are not contained in our vocabulary. If we can figure out a way to represent the unknown words, our model can still reason about whether they are a `LOCATION` or not, since we are also looking at the neighboring words for each prediction. 

We introduce a special token, `<unk>`, to tackle the words that are out of vocabulary. We could pick another string for our unknown token if we wanted. The only requirement here is that our token should be unique: we should only be using this token for unknown words. We will also add this special token to our vocabulary. 

In [68]:
# Add the unknown token to our vocabulary
vocabulary.add("<unk>")

Earlier we mentioned that our task was called `Word Window Classification` because our model is looking at the surroundings words in addition to the given word when it needs to make a prediction. 

For example, let's take the sentence "We always come to Paris". The corresponding training label for this sentence is `0, 0, 0, 0, 1` since only Paris, the last word, is a `LOCATION`. In one pass (meaning a call to `forward()`), our model will try to generate the correct label for one word. Let's say our model is trying to generate the correct label `1` for `Paris`. If we only allow our model to see `Paris`, but nothing else, we will miss out on the important information that the word `to` often times appears with `LOCATION`s. 

Word windows allow our model to consider the surrounding `+N` or `-N` words of each word when making a prediction. In our earlier example for `Paris`, if we have a window size of 1, that means our model will look at the words that come immediately before and after `Paris`, which are `to`, and, well, nothing. Now, this raises another issue. `Paris` is at the end of our sentence, so there isn't another word following it. Remember that we define the input dimensions of our `PyTorch` models when we are initializing them. If we set the window size to be `1`, it means that our model will be accepting `3` words in every pass. We cannot have our model expect `2` words from time to time.

The solution is to introduce a special token, such as `<pad>`, that will be added to our sentences to make sure that every word has a valid window around them. Similar to `<unk>` token, we could pick another string for our pad token if we wanted, as long as we make sure it is used for a unique purpose. 

In [69]:
# Add the <pad> token to our vocabulary
vocabulary.add("<pad>")

# Function that pads the given sentence
# We are introducing this function here as an example
# We will be utilizing it later in the tutorial
def pad_window(sentence, window_size, pad_token="<pad>"):
  window = [pad_token] * window_size
  return window + sentence + window

# Show padding example
window_size = 2
pad_window(train_sentences[0], window_size=window_size)

['<pad>',
 '<pad>',
 'auto',
 ',',
 'sun',
 'roof',
 ',',
 'lane',
 'assist',
 ',',
 'park',
 'assist,',
 'loaded,',
 'all',
 'apps',
 ',',
 'back',
 'up',
 'camera',
 ',',
 'navigation',
 ',',
 'bluetooth',
 ',',
 'super',
 'low',
 'miles',
 'with',
 'carfax',
 'to',
 'back',
 'up',
 ',',
 'non',
 'smoker',
 'super',
 'clean',
 'in',
 'and',
 'out',
 'salvage',
 'do',
 'to',
 'damage',
 'to',
 'rear',
 'bumper',
 'only',
 ',',
 'i',
 'included',
 'a',
 'before',
 'picture',
 'so',
 'u',
 'can',
 'se',
 'for',
 'your',
 'self',
 'car',
 'runs',
 'and',
 'looks',
 'lk',
 'new',
 'and',
 'it',
 'comes',
 'with',
 'a',
 '30',
 'day',
 'warranty',
 'and',
 '3',
 'day',
 'money',
 'back',
 'guaranteed',
 'for',
 'more',
 'info',
 'please',
 'call',
 'show',
 'contact',
 'info',
 '2017',
 'toyota',
 'prius',
 'loadedvin:',
 'jtdkarfu3h3045352',
 'condition:',
 'like',
 'new',
 'cylinders:',
 '4',
 'cylinders',
 'fuel:',
 'hybrid',
 'odometer:',
 '12332',
 'title',
 'status:',
 'salvage',
 't

Now that our vocabularly is ready, let's assign an index to each of our words. 

In [70]:
# We are just converting our vocabularly to a list to be able to index into it
# Sorting is not necessary, we sort to show an ordered word_to_ind dictionary
# That being said, we will see that having the index for the padding token
# be 0 is convenient as some PyTorch functions use it as a default value
# such as nn.utils.rnn.pad_sequence, which we will cover in a bit
ix_to_word = sorted(list(vocabulary))

# Creating a dictionary to find the index of a given word
word_to_ix = {word: ind for ind, word in enumerate(ix_to_word)}
word_to_ix

{'$13,900': 0,
 '$16,999': 1,
 '$22,000': 2,
 '$24,950': 3,
 '$31,000': 4,
 '$36500.fully': 5,
 '$900;': 6,
 '&': 7,
 '(': 8,
 '($2200)': 9,
 '($2400)': 10,
 '($7000': 11,
 '(2000': 12,
 '(4': 13,
 '(4-wheel)': 14,
 '(around': 15,
 '(common': 16,
 '(current': 17,
 '(feel': 18,
 '(for': 19,
 '(less': 20,
 '(listed': 21,
 '(minor': 22,
 '(motul)': 23,
 '(new': 24,
 '(nmo': 25,
 '(no': 26,
 '(not': 27,
 '(oem)': 28,
 '(p44),': 29,
 '(single': 30,
 '(stock': 31,
 '(v4),': 32,
 '(without': 33,
 ')': 34,
 '*': 35,
 '*2017+': 36,
 '*4': 37,
 '*8': 38,
 '*free': 39,
 '*standard-range': 40,
 ',': 41,
 ',call': 42,
 '-': 43,
 '-all': 44,
 '-blind': 45,
 '-brand': 46,
 '-carbon': 47,
 '-coil': 48,
 '-coolant': 49,
 '-dual': 50,
 '-factory': 51,
 '-fresh': 52,
 '-front': 53,
 '-ice': 54,
 '-maf': 55,
 '-magnetic': 56,
 '-oem': 57,
 '-pioneer': 58,
 '-previous': 59,
 '-push': 60,
 '-rear': 61,
 '-seat': 62,
 '-spark': 63,
 '-spt': 64,
 '-stage': 65,
 '-the': 66,
 '-timing': 67,
 '-transmission/fron

In [71]:
ix_to_word[1]

'$16,999'

Great! We are ready to convert our training sentences into a sequence of indices corresponding to each token. 

In [72]:
# Given a sentence of tokens, return the corresponding indices
def convert_token_to_indices(sentence, word_to_ix):
  indices = []
  for token in sentence:
    # Check if the token is in our vocabularly. If it is, get it's index. 
    # If not, get the index for the unknown token.
    if token in word_to_ix:
      index = word_to_ix[token]
    else:
      index = word_to_ix["<unk>"]
    indices.append(index)
  return indices

# More compact version of the same function
def _convert_token_to_indices(sentence, word_to_ix):
  return [word_to_ind.get(token, word_to_ix["<unk>"]) for token in sentence]

# Show an example
example_sentence = ["we", "always", "come", "to", "kuwait"]
example_indices = convert_token_to_indices(example_sentence, word_to_ix)
restored_example = [ix_to_word[ind] for ind in example_indices]

print(f"Original sentence is: {example_sentence}")
print(f"Going from words to indices: {example_indices}")
print(f"Going from indices to words: {restored_example}")

Original sentence is: ['we', 'always', 'come', 'to', 'kuwait']
Going from words to indices: [1657, 306, 249, 1552, 249]
Going from indices to words: ['we', 'always', '<unk>', 'to', '<unk>']


In the example above, `kuwait` shows up as `<unk>`, because it is not included in our vocabulary. Let's convert our `train_sentences` to `example_padded_indices`. 

In [73]:
# Converting our sentences to indices
example_padded_indices = [convert_token_to_indices(s, word_to_ix) for s in train_sentences]
example_padded_indices

[[350,
  41,
  1482,
  1328,
  41,
  938,
  336,
  41,
  1173,
  337,
  970,
  297,
  328,
  41,
  369,
  1604,
  461,
  41,
  1088,
  41,
  414,
  41,
  1485,
  979,
  1031,
  1688,
  474,
  1552,
  369,
  1604,
  41,
  1105,
  1434,
  1485,
  508,
  873,
  317,
  1147,
  1355,
  631,
  1552,
  594,
  1552,
  1283,
  443,
  1134,
  41,
  865,
  877,
  250,
  394,
  1201,
  1437,
  1598,
  464,
  1371,
  741,
  1712,
  1392,
  466,
  1345,
  317,
  978,
  968,
  1094,
  317,
  905,
  524,
  1688,
  250,
  156,
  599,
  1651,
  317,
  150,
  599,
  1055,
  369,
  799,
  741,
  1065,
  883,
  1209,
  458,
  1417,
  545,
  883,
  123,
  1565,
  1240,
  971,
  915,
  540,
  961,
  1094,
  591,
  172,
  590,
  756,
  864,
  1119,
  89,
  1549,
  1465,
  1355,
  1576,
  351],
 [1242,
  1178,
  1353,
  1549,
  873,
  808,
  508,
  476,
  128,
  1688,
  1134,
  246,
  1033,
  1189,
  538,
  478,
  107,
  951,
  933,
  1688,
  750,
  1415,
  317,
  478,
  1458,
  168,
  1564,
  1076,
  1131,
  

Now that we have an index for each word in our vocabularly, we can create an embedding table with `nn.Embedding` class in `PyTorch`. It is called as follows `nn.Embedding(num_words, embedding_dimension)` where `num_words` is the number of words in our vocabulary and the `embedding_dimension` is the dimension of the embeddings we want to have. There is nothing fancy about `nn.Embedding`: it is just a wrapper class around a trainabe `NxE` dimensional tensor, where `N` is the number of words in our vocabulary and `E` is the number of embedding dimensions. This table is initially random, but it will change over time. As we train our network, the gradients will be backpropagated all the way to the embedding layer, and hence our word embeddings would be updated. We will initiliaze the embedding layer we will use for our model in our model, but we are showing an example here. 

In [74]:
# Creating an embedding table for our words
embedding_dim = 5
embeds = nn.Embedding(len(vocabulary), embedding_dim)

# Printing the parameters in our embedding table
list(embeds.parameters())

[Parameter containing:
 tensor([[-0.0969, -0.2260, -0.3307, -1.2367, -0.4274],
         [ 0.0670,  0.5206, -0.2872,  1.0067,  0.4073],
         [-0.1209, -1.1162, -0.3176, -1.2438, -1.5316],
         ...,
         [-0.1035, -0.6344, -1.1732,  0.1575, -0.4824],
         [-0.7281, -1.6180,  0.9281,  1.5237,  0.0727],
         [ 0.6022, -0.3115, -2.5631, -1.2547, -0.4675]], requires_grad=True)]

To get the word embedding for a word in our vocabulary, all we need to do is to create a lookup tensor. The lookup tensor is just a tensor containing the index we want to look up `nn.Embedding` class expects an index tensor that is of type Long Tensor, so we should create our tensor accordingly. 

In [76]:
# Get the embedding for the word Paris
index = word_to_ix["honda"]
index_tensor = torch.tensor(index, dtype=torch.long)
paris_embed = embeds(index_tensor)
paris_embed

tensor([ 0.6890,  1.8147, -1.6392, -0.2573, -0.5227],
       grad_fn=<EmbeddingBackward>)

In [77]:
# We can also get multiple embeddings at once
index_honda = word_to_ix["honda"]
index_ford = word_to_ix["ford"]
indices = [index_honda, index_ford]
indices_tensor = torch.tensor(indices, dtype=torch.long)
embeddings = embeds(indices_tensor)
embeddings

tensor([[ 0.6890,  1.8147, -1.6392, -0.2573, -0.5227],
        [ 0.5883, -0.6978,  1.6093, -0.5106,  0.6829]],
       grad_fn=<EmbeddingBackward>)

Usually, we define the embedding layer as part of our model, which you will see in the later sections of our notebook. 

#### Batching Sentences

We have learned about batches in class. Waiting our whole training corpus to be processed before making an update is constly. On the other hand, updating the parameters after every training example causes the loss to be less stable between updates. To combat these issues, we instead update our parameters after training on a batch of data. This allows us to get a better estimate of the gradient of the global loss. In this section, we will learn how to structure our data into batches using the `torch.util.data.DataLoader` class. 

We will be calling the `DataLoader` class as follows: `DataLoader(data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)`.  The `batch_size` parameter determines the number of examples per batch. In every epoch, we will be iterating over all the batches using the `DataLoader`. The order of batches is deterministic by default, but we can ask `DataLoader` to shuffle the batches by setting the `shuffle` parameter to `True`. This way we ensure that we don't encounter a bad batch multiple times.

If provided, `DataLoader` passes the batches it prepares to the `collate_fn`. We can write a custom function to pass to the `collate_fn` parameter in order to print stats about our batch or perform extra processing. In our case, we will use the `collate_fn` to:
1. Window pad our train sentences.
2. Convert the words in the training examples to indices.
3. Pad the training examples so that all the sentences and labels have the same length. Similarly, we also need to pad the labels. This creates an issue because when calculating the loss, we need to know the actual number of words in a given example. We will also keep track of this number in the function we pass to the `collate_fn` parameter.

Because our version of the `collate_fn` function will need to access to our `word_to_ix` dictionary (so that it can turn words into indices), we will make use of the `partial` function in `Python`, which passes the parameters we give to the function we pass it. 

In [78]:
from torch.utils.data import DataLoader
from functools import partial

def custom_collate_fn(batch, window_size, word_to_ix):
  # Break our batch into the training examples (x) and labels (y)
  # We are turning our x and y into tensors because nn.utils.rnn.pad_sequence
  # method expects tensors. This is also useful since our model will be
  # expecting tensor inputs. 
  x, y = zip(*batch)

  # Now we need to window pad our training examples. We have already defined a 
  # function to handle window padding. We are including it here again so that
  # everything is in one place.
  def pad_window(sentence, window_size, pad_token="<pad>"):
    window = [pad_token] * window_size
    return window + sentence + window

  # Pad the train examples.
  x = [pad_window(s, window_size=window_size) for s in x]

  # Now we need to turn words in our training examples to indices. We are
  # copying the function defined earlier for the same reason as above.
  def convert_tokens_to_indices(sentence, word_to_ix):
    return [word_to_ix.get(token, word_to_ix["<unk>"]) for token in sentence]

  # Convert the train examples into indices.
  x = [convert_tokens_to_indices(s, word_to_ix) for s in x]

  # We will now pad the examples so that the lengths of all the example in 
  # one batch are the same, making it possible to do matrix operations. 
  # We set the batch_first parameter to True so that the returned matrix has 
  # the batch as the first dimension.
  pad_token_ix = word_to_ix["<pad>"]

  # pad_sequence function expects the input to be a tensor, so we turn x into one
  x = [torch.LongTensor(x_i) for x_i in x]
  x_padded = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=pad_token_ix)

  # We will also pad the labels. Before doing so, we will record the number 
  # of labels so that we know how many words existed in each example. 
  lengths = [len(label) for label in y]
  lenghts = torch.LongTensor(lengths)

  y = [torch.LongTensor(y_i) for y_i in y]
  y_padded = nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=0)

  # We are now ready to return our variables. The order we return our variables
  # here will match the order we read them in our training loop.
  return x_padded, y_padded, lenghts  

This function seems long, but it really doesn't have to be. Check out the alternative version below where we remove the extra function declarations and comments. 

In [79]:
def _custom_collate_fn(batch, window_size, word_to_ix):
  # Prepare the datapoints
  x, y = zip(*batch)  
  x = [pad_window(s, window_size=window_size) for s in x]
  x = [convert_tokens_to_indices(s, word_to_ix) for s in x]

  # Pad x so that all the examples in the batch have the same size
  pad_token_ix = word_to_ix["<pad>"]
  x = [torch.LongTensor(x_i) for x_i in x]
  x_padded = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=pad_token_ix)

  # Pad y and record the length
  lengths = [len(label) for label in y]
  lenghts = torch.LongTensor(lengths)
  y = [torch.LongTensor(y_i) for y_i in y]
  y_padded = nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=0)

  return x_padded, y_padded, lenghts  

Now, we can see the `DataLoader` in action. 

In [80]:
# Parameters to be passed to the DataLoader
data = list(zip(train_sentences, train_labels))
batch_size = 2
shuffle = True
window_size = 2
collate_fn = partial(custom_collate_fn, window_size=window_size, word_to_ix=word_to_ix)

# Instantiate the DataLoader
loader = DataLoader(data, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)

# Go through one loop
counter = 0
for batched_x, batched_y, batched_lengths in loader:
  print(f"Iteration {counter}")
  print("Batched Input:")
  print(batched_x)
  print("Batched Labels:")
  print(batched_y)
  print("Batched Lengths:")
  print(batched_lengths)
  print("")
  counter += 1

Iteration 0
Batched Input:
tensor([[ 248,  248,  123,  558, 1714,  171, 1716,  253,  333, 1677, 1688, 1299,
          892, 1716, 1163,   56, 1391, 1319,  547,   47,  488,  429,   65,  150,
          280, 1160,  873,  472,  731, 1669, 1169,  873,  472,  731,  230, 1094,
         1261,    9,  199,  488,  515, 1688, 1167,  557,  312, 1262,  826,   80,
           10,  431, 1094,  262,  708,  488, 1210, 1686,  431, 1094, 1098,  984,
         1447, 1211,  281, 1053,  889, 1589,  451,  777, 1697, 1071, 1193,  597,
         1298,  327,  481,    7,  318,  350,  833,  317,  552, 1382,  290, 1581,
          317,  831, 1611, 1046,  624,   16,  955,  900, 1403,  451,  254,  501,
           36,  501,  730,  553,  902, 1688, 1094, 1486,  953,  317,  274,  357,
         1265,  847, 1581,  360, 1239,  538,  508,  474,  361, 1504,  990,  991,
          992,  993,  314,  452,  453,  662,  663, 1064,  840,  559,  981,  982,
          577,  343,  422,  795, 1411,  792,  794, 1363, 1159,  169,  494, 1487,
 

The batched input tensors you see above will be passed into our model. On the other hand, we started off saying that our model will be a window classifier. The way our input tensors are currently formatted, we have all the words in a sentence in one datapoint. When we pass this input to our model, it needs to create the windows for each word, make a prediction as to whether the center word is a `LOCATION` or not for each window, put the predictions together and return. 

We could avoid this problem if we formatted our data by breaking it into windows beforehand. In this example, we will instead how our model take care of the formatting. 

Given that our `window_size` is `N` we want our model to make a prediction on every `2N+1` tokens. That is, if we have an input with `9` tokens, and a `window_size` of `2`, we want our model to return `5` predictions. This makes sense because before we padded it with `2` tokens on each side, our input also had `5` tokens in it! 

We can create these windows by using for loops, but there is a faster `PyTorch` alternative, which is the `unfold(dimension, size, step)` method. We can create the windows we need using this method as follows:

In [81]:
# Print the original tensor
print(f"Original Tensor: ")
print(batched_x)
print("")

# Create the 2 * 2 + 1 chunks
chunk = batched_x.unfold(1, window_size*2 + 1, 1)
print(f"Windows: ")
print(chunk)

Original Tensor: 
tensor([[ 248,  248,  118,  742,  668, 1453,  364,   43, 1594,  405,  155,  351,
         1572, 1688, 1000, 1165, 1413, 1138,  861, 1624, 1483, 1220,  693, 1531,
          668, 1653, 1142, 1688,  297, 1525,  396,  317, 1676,  943, 1382,    8,
          424,  648,  317, 1181, 1382,  329, 1220,  317,  834,  882,  510,  550,
          741,  648,  317, 1181, 1221, 1283, 1382,    8, 1529,  738,  642,  341,
         1525, 1557, 1121,  250,  447,  752, 1525, 1283,  477,  330,   34, 1308,
         1463, 1308,  954,  767,  781, 1088, 1557, 1369,  544, 1442, 1445, 1500,
         1358, 1266, 1279,    8,  916, 1091, 1712, 1480,   34,  410, 1456, 1056,
          861,  132, 1669, 1688, 1203, 1367, 1543, 1562,  850, 1293,    8,  957,
          659, 1297, 1688, 1525,  979, 1244, 1546,  865, 1248,  250, 1429, 1509,
          654, 1571,   34,  148, 1156, 1108,  466,  916, 1311,  382, 1212,  517,
         1164,  118,  742,  668, 1455,  695,  591,  207,  590,  645,  195,  756,
          

### Model

Now that we have prepared our data, we are ready to build our model. We have learned how to write custom `nn.Module` classes. We will do the same here and put everything we have learned so far together. 

In [82]:
class WordWindowClassifier(nn.Module):

  def __init__(self, hyperparameters, vocab_size, pad_ix=0):
    super(WordWindowClassifier, self).__init__()
    
    """ Instance variables """
    self.window_size = hyperparameters["window_size"]
    self.embed_dim = hyperparameters["embed_dim"]
    self.hidden_dim = hyperparameters["hidden_dim"]
    self.freeze_embeddings = hyperparameters["freeze_embeddings"]

    """ Embedding Layer 
    Takes in a tensor containing embedding indices, and returns the 
    corresponding embeddings. The output is of dim 
    (number_of_indices * embedding_dim).

    If freeze_embeddings is True, set the embedding layer parameters to be
    non-trainable. This is useful if we only want the parameters other than the
    embeddings parameters to change. 

    """
    self.embeds = nn.Embedding(vocab_size, self.embed_dim, padding_idx=pad_ix)
    if self.freeze_embeddings:
      self.embed_layer.weight.requires_grad = False

    """ Hidden Layer
    """
    full_window_size = 2 * window_size + 1
    self.hidden_layer = nn.Sequential(
      nn.Linear(full_window_size * self.embed_dim, self.hidden_dim), 
      nn.Tanh()
    )

    """ Output Layer
    """
    self.output_layer = nn.Linear(self.hidden_dim, 1)

    """ Probabilities 
    """
    self.probabilities = nn.Sigmoid()

  def forward(self, inputs):
    """
    Let B:= batch_size
        L:= window-padded sentence length
        D:= self.embed_dim
        S:= self.window_size
        H:= self.hidden_dim
        
    inputs: a (B, L) tensor of token indices
    """
    B, L = inputs.size()

    """
    Reshaping.
    Takes in a (B, L) LongTensor
    Outputs a (B, L~, S) LongTensor
    """
    # Fist, get our word windows for each word in our input.
    token_windows = inputs.unfold(1, 2 * self.window_size + 1, 1)
    _, adjusted_length, _ = token_windows.size()

    # Good idea to do internal tensor-size sanity checks, at the least in comments!
    assert token_windows.size() == (B, adjusted_length, 2 * self.window_size + 1)

    """
    Embedding.
    Takes in a torch.LongTensor of size (B, L~, S) 
    Outputs a (B, L~, S, D) FloatTensor.
    """
    embedded_windows = self.embeds(token_windows)

    """
    Reshaping.
    Takes in a (B, L~, S, D) FloatTensor.
    Resizes it into a (B, L~, S*D) FloatTensor.
    -1 argument "infers" what the last dimension should be based on leftover axes.
    """
    embedded_windows = embedded_windows.view(B, adjusted_length, -1)

    """
    Layer 1.
    Takes in a (B, L~, S*D) FloatTensor.
    Resizes it into a (B, L~, H) FloatTensor
    """
    layer_1 = self.hidden_layer(embedded_windows)

    """
    Layer 2
    Takes in a (B, L~, H) FloatTensor.
    Resizes it into a (B, L~, 1) FloatTensor.
    """
    output = self.output_layer(layer_1)

    """
    Softmax.
    Takes in a (B, L~, 1) FloatTensor of unnormalized class scores.
    Outputs a (B, L~, 1) FloatTensor of (log-)normalized class scores.
    """
    output = self.probabilities(output)
    output = output.view(B, -1)

    return output

### Training

We are now ready to put everything together. Let's start with preparing our data and intializing our model. We can then intialize our optimizer and define our loss function. This time, instead of using one of the predefined loss function as we did before, we will define our own loss function. 

In [83]:
# Prepare the data
data = list(zip(train_sentences, train_labels))
batch_size = 2
shuffle = True
window_size = 2
collate_fn = partial(custom_collate_fn, window_size=window_size, word_to_ix=word_to_ix)

# Instantiate a DataLoader
loader = DataLoader(data, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)

# Initialize a model
# It is useful to put all the model hyperparameters in a dictionary
model_hyperparameters = {
    "batch_size": 4,
    "window_size": 2,
    "embed_dim": 25,
    "hidden_dim": 25,
    "freeze_embeddings": False,
}

vocab_size = len(word_to_ix)
model = WordWindowClassifier(model_hyperparameters, vocab_size)

# Define an optimizer
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Define a loss function, which computes to binary cross entropy loss
def loss_function(batch_outputs, batch_labels, batch_lengths):   
    # Calculate the loss for the whole batch
    bceloss = nn.BCELoss()
    loss = bceloss(batch_outputs, batch_labels.float())

    # Rescale the loss. Remember that we have used lengths to store the 
    # number of words in each training example
    loss = loss / batch_lengths.sum().float()

    return loss

Unlike our earlier example, this time instead of passing all of our training data to the model at once in each epoch, we will be utilizing batches. Hence, in each training epoch iteration, we also iterate over the batches.

In [84]:
# Function that will be called in every epoch
def train_epoch(loss_function, optimizer, model, loader):
  
  # Keep track of the total loss for the batch
  total_loss = 0
  for batch_inputs, batch_labels, batch_lengths in loader:
    # Clear the gradients
    optimizer.zero_grad()
    # Run a forward pass
    outputs = model.forward(batch_inputs)
    # Compute the batch loss
    loss = loss_function(outputs, batch_labels, batch_lengths)
    # Calculate the gradients
    loss.backward()
    # Update the parameteres
    optimizer.step()
    total_loss += loss.item()

  return total_loss


# Function containing our main training loop
def train(loss_function, optimizer, model, loader, num_epochs=10000):

  # Iterate through each epoch and call our train_epoch function
  for epoch in range(num_epochs):
    epoch_loss = train_epoch(loss_function, optimizer, model, loader)
    if epoch % 100 == 0: print(epoch_loss)

Let's start training!

In [85]:
num_epochs = 1000
train(loss_function, optimizer, model, loader, num_epochs=num_epochs)

0.05585021816659719
0.04946008767001331
0.04739081102889031
0.05051335925236344
0.04657275672070682
0.04191842454019934
0.03870241594268009
0.03348514199024066
0.03834902576636523
0.039742261345963925


### Prediction

Let's see how well our model is at making predictions. We can start by creating our test data.

In [88]:
# Create test sentences
test_corpus = ["ford f-150 platinum crew cab 4x4 if you are looking for a truck in perfect condition this is it! it has every factory option available in 2015 contact me for specifics. No accidents very clean inside and out. garaged since day one detailed weekly every service has been performed on time. the only reason I am selling it is I bought a new one. Title in hand no low ballers or dealers please. Serious inquiries only."]
test_sentences = [s.lower().split() for s in test_corpus]
test_labels = [[0, 0, 0, 1]]

# Create a test loader
test_data = list(zip(test_sentences, test_labels))
batch_size = 1
shuffle = False
window_size = 2
collate_fn = partial(custom_collate_fn, window_size=2, word_to_ix=word_to_ix)
test_loader = torch.utils.data.DataLoader(test_data, 
                                           batch_size=1, 
                                           shuffle=False, 
                                           collate_fn=collate_fn)

Let's loop over our test examples to see how well we are doing. 

In [89]:
for test_instance, labels, _ in test_loader:
  outputs = model.forward(test_instance)
  print(labels)
  print(outputs)

tensor([[0, 0, 0, 1]])
tensor([[0.3263, 0.4940, 0.5099, 0.5251, 0.3273, 0.3771, 0.4018, 0.3977, 0.3515,
         0.4045, 0.2856, 0.3260, 0.3794, 0.2975, 0.3903, 0.4543, 0.4668, 0.3153,
         0.4074, 0.4646, 0.3675, 0.4863, 0.3722, 0.4030, 0.4833, 0.3132, 0.4738,
         0.2491, 0.5403, 0.3694, 0.4625, 0.3667, 0.3647, 0.3912, 0.3656, 0.3792,
         0.3633, 0.4481, 0.4192, 0.4565, 0.5616, 0.5511, 0.5205, 0.4360, 0.4037,
         0.2462, 0.5269, 0.4593, 0.4563, 0.4659, 0.5304, 0.5025, 0.4037, 0.4748,
         0.3571, 0.4237, 0.3756, 0.4457, 0.4009, 0.3523, 0.3770, 0.5313, 0.4058,
         0.3860, 0.4537, 0.4321, 0.3779, 0.4978, 0.3483, 0.5082, 0.5116, 0.3471,
         0.5665, 0.5273, 0.2726, 0.2326]], grad_fn=<ViewBackward>)


In [46]:
torch.save(model, 'models/model.pt' )

In [47]:
saved_model = torch.load('models/model.pt')

In [93]:
outputs

[tensor([0.3263, 0.4940, 0.5099, 0.5251, 0.3273, 0.3771, 0.4018, 0.3977, 0.3515,
         0.4045, 0.2856, 0.3260, 0.3794, 0.2975, 0.3903, 0.4543, 0.4668, 0.3153,
         0.4074, 0.4646, 0.3675, 0.4863, 0.3722, 0.4030, 0.4833, 0.3132, 0.4738,
         0.2491, 0.5403, 0.3694, 0.4625, 0.3667, 0.3647, 0.3912, 0.3656, 0.3792,
         0.3633, 0.4481, 0.4192, 0.4565, 0.5616, 0.5511, 0.5205, 0.4360, 0.4037,
         0.2462, 0.5269, 0.4593, 0.4563, 0.4659, 0.5304, 0.5025, 0.4037, 0.4748,
         0.3571, 0.4237, 0.3756, 0.4457, 0.4009, 0.3523, 0.3770, 0.5313, 0.4058,
         0.3860, 0.4537, 0.4321, 0.3779, 0.4978, 0.3483, 0.5082, 0.5116, 0.3471,
         0.5665, 0.5273, 0.2726, 0.2326], grad_fn=<UnbindBackward>)]

In [94]:
test_sentences

[['ford',
  'f-150',
  'platinum',
  'crew',
  'cab',
  '4x4',
  'if',
  'you',
  'are',
  'looking',
  'for',
  'a',
  'truck',
  'in',
  'perfect',
  'condition',
  'this',
  'is',
  'it!',
  'it',
  'has',
  'every',
  'factory',
  'option',
  'available',
  'in',
  '2015',
  'contact',
  'me',
  'for',
  'specifics.',
  'no',
  'accidents',
  'very',
  'clean',
  'inside',
  'and',
  'out.',
  'garaged',
  'since',
  'day',
  'one',
  'detailed',
  'weekly',
  'every',
  'service',
  'has',
  'been',
  'performed',
  'on',
  'time.',
  'the',
  'only',
  'reason',
  'i',
  'am',
  'selling',
  'it',
  'is',
  'i',
  'bought',
  'a',
  'new',
  'one.',
  'title',
  'in',
  'hand',
  'no',
  'low',
  'ballers',
  'or',
  'dealers',
  'please.',
  'serious',
  'inquiries',
  'only.']]